In [1]:
import torch
import torch.nn as nn
from vit_pytorch import ViT
from graphs import Graph, prims
import os
import numpy as np
import torchvision.models as models
import torchvision.transforms as transforms
from torchvision.models import ResNet18_Weights
from tree_dataset import TreeDataset
import model as m
from torch.utils.data import DataLoader
from d2lvit import *
import copy
from collections import OrderedDict

In [2]:
model = torch.load(os.path.join('..', 'models', 'finetune_3', 'digit-model.pt'))

In [3]:
train_set = TreeDataset(os.path.join('..', 'data', 'extra_variety_4k'), m.resnet_preprocess()) 
val_set = TreeDataset(os.path.join('..', 'data', 'extra_variety_2k'), m.resnet_preprocess()) 
print(f'Train size: {len(train_set)} Val size: {len(val_set)}')
train_loader = DataLoader(train_set, batch_size=32)
val_loader = DataLoader(val_set, batch_size=32)
device = m.get_device()
config = {'labels_key': 'digit_labels'}

Train size: 2000 Val size: 1000
Identified CUDA device: NVIDIA GeForce RTX 3060


In [4]:
train_acc = m.predict(model, train_loader, device, config, None)
print(train_acc)

1.0


In [5]:
val_acc = m.predict(model, val_loader, device, config, None)
print(val_acc)

1.0


In [6]:
digits_model = copy.deepcopy(model)

In [7]:
model.heads = nn.Sequential(OrderedDict([('head', nn.Linear(768, 45))]))
model = model.to(device)

In [8]:
model(torch.unsqueeze(train_set[0]['image'], 0).to(device))

tensor([[ 0.2506, -0.4085, -0.8244,  0.2150,  0.5123, -0.3016,  0.1497, -0.6505,
          0.5827,  0.2583, -0.2379, -0.1441,  1.0354, -0.4329,  0.4975, -0.8745,
         -0.0526,  0.6005,  0.6472, -0.3056, -0.0255,  0.4499, -0.1670,  0.6950,
          0.5617, -0.4099,  0.8095, -0.1273, -0.0893, -0.0333, -0.1000,  0.1706,
          0.1084,  0.9215, -0.0618, -0.1876,  0.6713, -0.8055,  0.1535,  0.0687,
          0.3582, -0.5396, -0.0336,  0.1627,  1.1198]], device='cuda:0',
       grad_fn=<AddmmBackward0>)

In [9]:
m.train(model, 0.0001, 0, 100, train_loader, val_loader, device, os.path.join('..', 'models', 'finetune_noprims'), digits_model)

Epoch 10 done, train loss: 0.0023 val acc: 1.0000
Epoch 20 done, train loss: 0.0007 val acc: 1.0000
Epoch 30 done, train loss: 0.0003 val acc: 1.0000
Epoch 40 done, train loss: 0.0002 val acc: 1.0000
Epoch 50 done, train loss: 0.0001 val acc: 1.0000
Epoch 60 done, train loss: 0.0001 val acc: 1.0000
Epoch 70 done, train loss: 0.0000 val acc: 1.0000
Epoch 80 done, train loss: 0.0000 val acc: 1.0000


KeyboardInterrupt: 

In [ ]:
model = torch.load(os.path.join('..', 'models', 'finetune_noprims', 'tree-model.pt'))

In [ ]:
config = {'labels_key': 'tree_label'}
train_acc = m.predict(model, train_loader, device, config, digits_model)
print(train_acc)

In [ ]:
val_acc = m.predict(model, val_loader, device, config, digits_model)
print(val_acc)